In [12]:
# Transcribe a PDF file 

import PyPDF2

def pdf_to_txt(input_path, output_path):
    with open(input_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        with open(output_path, 'w') as output_file:
            for page_num in range(num_pages):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                output_file.write(text)

    print('Transcription completed successfully.')

# Example usage
pdf_path = '0Ecologia.pdf'
txt_path = 'output.txt'
pdf_to_txt(pdf_path, txt_path)

Transcription completed successfully.


In [ ]:
# Now we are going to use OpenAI to summarize this document

In [ ]:
import PyPDF2
import openai

# Configure your OpenAI API credentials
openai.api_key = ''

def pdf_to_summary(input_path):
    with open(input_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        transcription = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            transcription += text

    prompt = "Summarize the content about Ecology contained in the following text:\n" + transcription

    # Send the prompt to OpenAI for summarization
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=100,
        temperature=0.3,
        n=1,
        stop=None
    )

    summary = response.choices[0].text.strip()

    # Write the summary to a TXT file
    with open("summary.txt", "w") as output_file:
        output_file.write(summary)

    print("Summary generated and saved in 'summary.txt'.")

# Example usage
pdf_path = "C:/Users/luisg/Downloads/0Ecologia.pdf"
pdf_to_summary(pdf_path)


In [ ]:
import openai
import os
from time import time, sleep
import textwrap
import re


def open_file(filepath, encoding='utf-8'):
    with open(filepath, 'r', encoding=encoding, errors='ignore') as infile:
        return infile.read()


openai.api_key = ''


def save_file(content, filepath):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)


def gpt3_completion(prompt, engine='text-davinci-002', temp=0.6, top_p=1.0, tokens=2000, freq_pen=0.25, pres_pen=0.0, stop=['<<END>>']):
    max_retry = 5
    retry = 0
    while True:
        try:
            response = openai.Completion.create(
                engine=engine,
                prompt=prompt,
                temperature=temp,
                max_tokens=tokens,
                top_p=top_p,
                frequency_penalty=freq_pen,
                presence_penalty=pres_pen,
                stop=stop)
            text = response['choices'][0]['text'].strip()
            text = re.sub('\s+', ' ', text)
            filename = '%s_gpt3.txt' % time()
            with open('gpt3_logs/%s' % filename, 'w') as outfile:
                outfile.write('PROMPT:\n\n' + prompt + '\n\n==========\n\nRESPONSE:\n\n' + text)
            return text
        except Exception as oops:
            retry += 1
            if retry >= max_retry:
                return "GPT3 error: %s" % oops
            print('Error communicating with OpenAI:', oops)
            sleep(1)


if __name__ == '__main__':
    input_path = "C:/Users/luisg/Downloads/0Ecologia.pdf"
    output_path = "C:/Users/luisg/Downloads/summary.txt"

    alltext = open_file(input_path)
    chunks = textwrap.wrap(alltext, 2000)
    result = []
    count = 0
    for chunk in chunks:
        count += 1
        prompt = open_file('prompt.txt').replace('<<SUMMARY>>', chunk)
        prompt = prompt.encode(encoding='ASCII', errors='ignore').decode()
        summary = gpt3_completion(prompt)
        print('\n\n\n', count, 'of', len(chunks), ' - ', summary)
        result.append(summary)
    save_file('\n\n'.join(result), output_path)
